In [1]:
import pandas as pd
import jieba
import jieba.analyse
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

pd.set_option('display.max_colwidth', None)#setting the maximize string show

In [2]:
df = pd.read_csv("KCC Data/NewsCisDepAll_Seg-35325.csv", encoding='utf-8-sig') 

In [3]:
df.head(2)

,編號,質詢單位,詳細,逐字稿
0,15601,陳美雅,2-1-1040504-2-陳美雅-1,陳議員美雅：本席的市政總質詢想針對高雄市民關心的許多議題就教市府團隊。首先本席要請教市長，高雄市目前對於大陸之間的兩岸關係，市長的態度如何，認為我們跟大陸之間的交流，你是支持還是反對？請市長表達你的立場。主席（康議長裕成）：市長，請回答。陳市長菊：高雄市是一個開放的城市，高雄市跟中國大陸之間，我們一向都是鼓勵交流。同時我個人為了2009年世界運動會、2013年亞太城市高峰會議，我都曾經到中國大陸相關的城市邀請他們來參與。我想做為一個開放的城市，我們都是…。陳議員美雅：所以市長你認為台灣目前應該維持什麼樣的政治立場，你目前的政治立場可不可以更簡單一點？陳市長菊：台灣現階段就是維持現狀。陳議員美雅：我們的現狀可不可以請市長用更白話一點告訴市民？陳市長菊：這跟市政無關，我想今天不討論政治議題。陳議員美雅：本席現在要了解的，是要跟高雄市民講，如果剛剛市長表達的立場是未來也希望增加兩岸之間的交流…。陳市長菊：我同意。陳議員美雅：那麼市長是否可以更坦然的面對這個問題，讓兩岸之間的交流和平發展？陳市長菊：當然。陳議員美雅：所以你認為目前台灣跟大陸之間朝著所謂的和平發展，是指九二共識你也認同的意思嗎？陳市長菊：兩岸之間朝著更進步的的發展，我想九二共識有不同的解讀，這個部分在台灣社會有不同的看法…。陳議員美雅：市長你的解讀呢？陳市長菊：我想這跟今天的市政無關。謝謝。陳議員美雅：這跟未來高雄市政，大陸未來如何跟高雄市之間做的觀光文化交流，甚至未來有沒有任何經貿交流有關，市長你好像一直迴避這個問題。陳市長菊：我們都贊成跟中國大陸之間不斷的開放交流，藉由民間更多的交流，異中求同，然後爭取更多彼此的認知及認識，這樣兩岸之間才能更趨和緩。謝謝。陳議員美雅：謝謝市長，市長剛剛的意思也尊重所謂的九二共識。陳市長菊：沒有，那是你的解讀，我不是這樣解讀的。陳議員美雅：市長，你的解讀讓你更白話的說一句話，你就用一句話表達你的立場，我的市政質詢時間非常寶貴。陳市長菊：我的答覆也是很寶貴，時間是共同的。陳議員美雅：所以你的立場到底是什麼呢？陳市長菊：我的立場很清楚，兩岸之間異中求同，兩岸之間有很多歷史發展的過程不一樣，兩岸之間應該要更多的往來、互相尊重、異中求同。謝謝。陳議員美雅：互相尊重嘛！謝謝你認同所謂的互相尊重這一點。我想高雄不要再有所謂的意識形態，議員在高雄市議會要為高雄市、為台灣表達一個立場，我們希望讓高雄市民能夠生活得更好，不要總是赤字困難。我們希望能創造高雄跟兩岸之間，甚至各國之間是友好的環境，讓大家樂於來高雄市投資，我們就是希望讓高雄市民未來有更多的就業機會，未來高雄市能自創更多的自有財源。這是市長未來應該要為高雄市民奮鬥努力的。
1,15602,陳美雅,2-1-1040504-2-陳美雅-10,陳議員美雅：請尊重本席的時間。市長，這就是我們的官員，新的局長也要來和議員嘔，難怪副市長做錯誤的示範，各位高雄市民，民進黨完全執政之下，居然就是這樣蠻橫的狀態，對於議員的質詢不針對問題回答，還要用回嗆的方式，甚至沒有請他回答時，他也要在台下用叫囂的方式。本席再請教市長，45億6


In [4]:
df['質詢單位'].unique()

array(['陳美雅', '李喬如', '簡煥宗', '蔡金晏', '國民黨團', '民進黨團'], dtype=object)

In [5]:
ly = df[df.質詢單位 == "簡煥宗"]

In [6]:
doc_names = range(len(ly)) #有幾篇文章
doc_names

range(0, 230)

In [7]:
text_list = list(ly['逐字稿'])
ly_docs = pd.DataFrame(columns=['jieba_results'])
ly_docs['jieba_results'] = ly_docs['jieba_results'].astype('str')

In [8]:
words = jieba.lcut(text_list[1], cut_all = False)
print(words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.662 seconds.
Prefix dict has been built successfully.


['接下', '來', '的', '時間', '，', '我會', '跟', '李議員', '喬如針', '對', '哈瑪星', '的', '交通', '問題', '做', '相關', '的', '質詢', '。', '李議員', '喬如', '：', '謝謝', '簡議員煥', '宗願意', '利用', '這麼', '寶貴', '的', '總質詢', '時間', '，', '提出', '鼓山', '、', '鹽', '埕', '、', '旗津', '的', '觀光', '交通', '問題', '，', '讓', '李喬如', '一起', '來', '努力', '協助', '。', '在', '質詢', '之前', '，', '我', '非常', '佩服', '，', '也', '感謝並', '肯定', '高議員', '閔琳', '及', '簡議員煥宗', '，', '今天', '總質詢', '的', '第一', '砲', '就', '針對', '兩性', '平權法', '和', '同志', '婚姻', '註', '記', '法', '，', '做', '一個', '歷史性', '的', '突破', '及', '解決', '，', '我', '相信', '相關', '的', '高雄市', '民和', '年', '輕人', '都', '有感', '覺', '的', '。']


In [9]:
punctuation = " //，：:""()\n!！？｡＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
re_punctuation = "[{}] ".format(punctuation)

#creat a df to store the jieba result
ly_docs = pd.DataFrame(columns=['jieba_results']) # 建立df
ly_docs['jieba_results'] = ly_docs['jieba_results'].astype('str')

jieba.set_dictionary("KCC Data/Dict/dict.big.txt") #更改預設辭典（dict.big為繁體版）
jieba.load_userdict("KCC Data/Dict/KccDict2020.txt") #領域辭典
jieba.analyse.set_stop_words("KCC Data/Dict/KccStopWord2020.txt")

stopword_list = []
with open("KCC Data/Dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords: #斷詞辭典
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

for i in doc_names: #從第一篇到最後一篇
    words = jieba.cut(text_list[i], cut_all = False)
    text = ''
    for word in words:
        text = text + ' ' + str(word) 
    #print(words)
    #print(i) #第幾篇文章
    #print(text) #第幾篇文章的斷詞結果
    text = re.sub(re_punctuation, "", text) # 把上面那些符號從text中刪掉
    text = re.sub(r'[0-9]','',text) # 刪掉數字
    text = re.sub(r'[a-zA-Z]','',text) #刪掉英文字
#刪除文字 更多標點符號
    text = re.sub(r'[-<>#。！⋯.➡?=&▶_%♀!❗🎉⏰💪㊙️✨🔺😁😅😍⬇🙏✅🎈🔥📱👍👊🔸📍👉📢👩📌💼]','',text)  # 注意裡面不能有空白！
    s = pd.Series({'jieba_results': text})
    ly_docs = ly_docs.append(s, ignore_index=True)#每次的斷詞結果都加回liu_docs

Building prefix dict from C:\Users\User\OneDrive\POLITICS-AND-INFORMATION-master\20210517_code\KCC Data\Dict\dict.big.txt ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ue06d503e3529cb2586146b6160a5167a.cache
Loading model cost 1.142 seconds.
Prefix dict has been built successfully.


In [10]:
ly_docs

,jieba_results
0,簡議員煥宗 我 是 來自 旗津 鼓山 鹽埕 的 市 議員簡煥宗 今天 是 我 當選 之後 第一次 總質詢 請教 市長 我們 年輕 這一輩 從事 公共事務 有 被 幾個 人 影響 到 其中 對 我 影響 最深 的 一位 就是 鄭南榕 先生 鄭南榕 先生 為了 言論自由 而 犧牲 他 的 生命 他 有 幾句 經典 名言 譬如說 我 是 鄭南榕 我 主張 台灣獨立 剩下 的 就是 你們 的 事情 了 等 最近 教育部 課程 審議 小組 針對 高 中 公民 課本 把 鄭南榕 先生 這 段 歷史 以及 白色恐怖 的 過程 他們 要 拿掉 請教 長期以來 為 民主 為 人權 打 拚 的 陳菊市長 針對 教育部 課程 審議 委員會 這樣 的 作為 高雄市政府 有 怎樣 的 態度 有 怎樣 的 意見 市長 請 回應 主席 蔡副議長昌達 市長 請答 覆 陳市長菊 昨天 是 母親節 剛好 是 林義雄 先生 慈林 紀念館 第二個 新館 獻館 所以 邀請 我 參加 林義雄 先生 年 發生 林家血案 年前 他 的 雙胞胎 女兒 和 他 的 母親 被 殺害 到 現在 都 是 一個 懸案 台灣 民主 在 發展 的 過程 之中 有 很多 人 犧牲生命 有 很多 人 家破人亡 有 很多 人 流亡 海內外 鄭南榕 先生 在 民主 發展 的 過程 年 美麗島事件 以後 他 一直 主張 百分之百 的 言論自由 最後 他用 非常 悲壯 的 自焚 方式 來 爭取 百分之百 的 言論自由 今天 他 是 我們 台灣 追求 新聞自由 言論自由 最 重要 的 一個 階段 在 這個 情況 之下 不管 你 喜歡 或 不 喜歡 歷史 的 事實 不容許 毀滅 不容許 抹煞 今天 教育部 的 課綱 一直 都 是 黑箱 作業 今天 不 只 一 般的 社會 包括 民進黨 執政 的 縣市 都 不能 接受 我 覺得 你 抹煞 台灣 共同 的 記憶 包括 民主 發展 的 過程 這種 抹煞 這種 對 歷史 的 壓制 竄改 我 認為 這是 非常 卑劣 的 行為 立法院 民進黨 黨團 包括 民進黨 中央 對 這種 課綱 的 黑箱 作業 希望 能夠 採取 一致 的 做法 我 認為 民主 發展 的 過程 包括 白色恐怖 不管 受害 的 任何人 任何人 受害 都 應該 在 現階段 我們 有 一個 正確 真實 的 紀錄 讓 下一代 能夠 記取 教訓 就 像 二次大戰 納粹 很多 劣跡 很多 對 猶太人 的 迫害 然後 我們 去 歐洲 就 看到 那些 集中營 我們 看 了 都 很 難過 不 知道 人類 社會 怎麼 會 這樣 做 他們 還是 保留 了 這個 事實 不會 因為 說 當時 德國人 這樣 做 今天 就 把 所有 那些 歷史 記憶 全部 毀滅 不是 他們 正面 去 面對 它 今天 台灣 社會 應該 用 更 寬容 更 寬大 面對 每一個 階段 歷史 發展 的 過程 所以 我們 不 允許 鄭南榕 先生 爭取 言論自由 他用 最 悲壯 的 自焚 方式 對 當時 的 台灣 社會 衝擊 很大 我們 認為 這 段 事實 不 應該 被 抹滅 簡議員煥宗 歷史 必須 要 被 記取 的 歷史 帶給 我們 很多 檢討 和 進步 的 空間 我 在 這邊 譴責 教育部 這種 黑箱 作業 把 鄭南榕 先生 和 白色恐怖 那 一段 歷史 抹 滅掉
1,接下來 的 時間 我會 跟 李議員喬如 針對 哈瑪星 的 交通問題 做 相關 的 質詢 李議員喬如 謝謝 簡議員煥宗 願意 利用 這麼 寶貴 的 總質詢 時間 提出 鼓山 鹽埕 旗津 的 觀光 交通問題 讓 李喬如 一 起來 努力 協助 在 質詢 之前 我 非常 佩服 也 感謝 並 肯定 高議員閔琳 及 簡議員煥宗 今天 總質詢 的 第一 砲 就 針對 兩性平權 法 和 同志 婚姻 註 記法 做 一個 歷史性 的 突破 及 解決 我 相信 相關 的 高雄市 民和 年輕人 都 有 感覺 的
2,有關 交通 的 部分 本席 跟 簡 議員 在 當選 之後 多次 針對 這個 問題 要 解決 但是 看起來 有些 大致 還 不如人意 今天 最 重要 的 是 要 市長 知道 一下 我們 兩位 議員 很 認真 努力 的 突破 但是 有些 部分 還 需要 市長 再 做 政策性 的 突破 鼓山 鹽埕 旗津 地區 因為 市長 非常 努力 觀光 地點 有山 有海 有河 在 市長 任內 付出 非常 多 的 建設 經費 讓 鼓山 鹽埕 旗津 的 觀光 這麼 蓬勃 這麼 發達 我們 不 反對 觀光 當然 觀光 蓬勃 之後 會 造成 交通 壅塞 及 環保 問題 市區 居民 就 會 像 上 的 抗議 我們 二位 議員 也 開 了 多次 公聽會 也 獻策 給 市政府 本席 和 簡議員煥宗 在 公聽會 也 提出 總量管制 交通局 也 做 了 微幅 修正 我 建議 中央補助 款 的 輛 觀光 大 巴士 要 進來 地方 反彈 原因 是 遊覽車 進入 造成 交通 壅塞 空氣污染 如果 你 還是 這樣 做 對 社區 來說 是 換湯不換藥 不會 有 改進 的 我們 也 希望 多次 的 公聽會 和 會 勘 我們 向 局處 提出 的 建言 在 旗津 地區 渡船場 有關 港務公司 往北 堤岸 的 部分 是不是 請 市長 出面 責成 哪些 單位 來 提供 擴建 計畫 選前 我們 會 勘過 高港 公司 不會 有 意見 市政府 如果 願意 提供 他們 願意 配合
3,接下來 的 時間 我會 跟 李議員喬如 針對 哈瑪星 的 交通問題 做 相關 的 質詢 李議員喬如 謝謝 簡議員煥宗 願意 利用 這麼 寶貴 的 總質詢 時間 提出 鼓山 鹽埕 旗津 的 觀光 交通問題 讓 李喬如 一 起來 努力 協助 在 質詢 之前 我 非常 佩服 也 感謝 並 肯定 高議員閔琳 及 簡議員煥宗 今天 總質詢 的 第一 砲 就 針對 兩性平權 法 和 同志 婚姻 註 記法 做 一個 歷史性 的 突破 及 解決 我 相信 相關 的 高雄市 民和 年輕人 都 有 感覺 的 有關 交通 的 部分 本席 跟 簡 議員 在 當選 之後 多次 針對 這個 問題 要 解決 但是 看起來 有些 大致 還 不如人意 今天 最 重要 的 是 要 市長 知道 一下 我們 兩位 議員 很 認真 努力 的 突破 但是 有些 部分 還 需要 市長 再 做 政策性 的 突破 鼓山 鹽埕 旗津 地區 因為 市長 非常 努力 觀光 地點 有山 有海 有河 在 市長 任內 付出 非常 多 的 建設 經費 讓 鼓山 鹽埕 旗津 的 觀光 這麼 蓬勃 這麼 發達 我們 不 反對 觀光 當然 觀光 蓬勃 之後 會 造成 交通 壅塞 及 環保 問題 市區 居民 就 會 像 上 的 抗議 我們 二位 議員 也 開 了 多次 公聽會 也 獻策 給 市政府 本席 和 簡議員煥宗 在 公聽會 也 提出 總量管制 交通局 也 做 了 微幅 修正 我 建議 中央補助 款 的 輛 觀光 大 巴士 要 進來 地方 反彈 原因 是 遊覽車 進入 造成 交通 壅塞 空氣污染 如果 你 還是 這樣 做 對 社區 來說 是 換湯不換藥 不會 有 改進 的 我們 也 希望 多次 的 公聽會 和 會 勘 我們 向 局處 提出 的 建言 在 旗津 地區 渡船場 有關 港務公司 往北 堤岸 的 部分 是不是 請 市長 出面 責成 哪些 單位 來 提供 擴建 計畫 選前 我們 會 勘過 高港 公司 不會 有 意見 市政府 如果 願意 提供 他們 願意 配合 另外 簡議員煥宗 也 提出 一些 報告 哈瑪星渡船場 哨船頭 是 通往 鼓山 最 重要 的 幹道 與 交通量 非常 有關 對 於 高雄區漁會 的魚 市場 也 需要 市長 出面 跟 高雄區漁會 做 租借 或 土地 交換 因為 我們 認為 如果 這裡 沒有 改善 也 沒有 罰則 可以 疏散 人潮 的 處理 辦法 再 來 我們 也 建議 輪船 要 汰舊換新 噸位 真的 不足 了 希望 更新 之後 輪船 的 運輸量 可以 提升 也 希望 未來 接駁 的 觀光車 不是 汽油 的 而是 電動 的 這樣 的 管制 之下 社區 空氣品質 才能 維護 其他 部分 簡議員煥宗 有 更 多 更好 的 方式 一起 討論 簡議員煥宗 接下來 請 鼓山分局 的 曾 局長 進 到 議事廳 針對 這件 事情 做 討論 主席 蔡副議長昌達 分局長 請進來 簡議員煥宗 在 分局長 還未 進來 之前 我先 報告 這是 來自 交通局 的 簡報 說明 過去 平均 每天 有 輛次 遊覽車 進入 因為 社區 的 壓力 和 地方 上 的 輿情 其實 交通局 有 做一套 新 的 交通 輸運 方式 月 日要 開始 處理 可是 在還 未處理 之前 月 日 下午 點多到 點我 一直 在 那裡 我 看到 臨海 新路 也 就是 駁二 和 鐵道園區 的 路 整個 都 被 遊覽車 停滿 了 而且 是 下班時間 很 危險 因為 市民 朋友 的 交通工具 都 是 摩托車 鼓山分局 為了 執行 整個 任務 把 龍華 所 內惟所 鼓山 所 的 相關 同仁 都 調去 支援 很 辛苦 尤其 我 拍 到 一個 畫面 這部 遊覽車 一 併 排 警察 同仁 馬上 騎 機車 過去 勸導 我 真的 覺得 警察 同仁 很 辛苦 不 曉得 未來 交通局 針對 這部分 要 如何 改善 另外 還有 鄧麗君紀念館 也 面臨 同樣 的 問題 都 是 遊覽車 併 排 的 問題 我 想 請問 一下 鼓山分局 的 分局長 針對 遊覽車 併 排 的 部分 目前 開單 的 狀況 怎

In [11]:
ly_docs.to_csv('簡煥宗_output.txt')

In [12]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [13]:
fList = []
with open("簡煥宗_output.txt","r", encoding='utf-8-sig') as files:
     for file in files:
        #simple_preprocess: tokenize,去除標點 - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15) 
        fList.append(list(utils.simple_preprocess(file, deacc=True, min_len=2,)))
print("fList[:1] = ", fList[:1])
print("------------------------------")

fList[:1] =  [['jieba_results']]
------------------------------


In [14]:
dict = corpora.Dictionary(fList)
print("dict = ", dict)
print("------------------------------")

dict =  Dictionary(12446 unique tokens: ['jieba_results', '一位', '一個', '一段', '一直']...)
------------------------------


In [15]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in fList]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:1]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('jieba_results', 1)


In [16]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("簡煥宗_ldaoutput.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(fList), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.024*"淹水" + 0.019*"水利局" + 0.016*"地區" + 0.012*"鹽埕" + 0.011*"鹽埕區" + 0.011*"市場" + 0.011*"抽水站" + 0.009*"經發局" + 0.008*"滯洪池" + 0.008*"愛河" + 0.008*"改善" + 0.007*"經濟發展局曾局長文生" + 0.007*"防災" + 0.006*"閘門" + 0.006*"大潮" + 0.005*"抽水機" + 0.005*"鼓山三路" + 0.005*"鼓山" + 0.005*"倒灌" + 0.005*"公分"

strTM =  10.031*"學校" + 0.025*"教育局" + 0.023*"美術館" + 0.014*"鐵路地下化" + 0.014*"中山國小" + 0.010*"工務局" + 0.009*"未來" + 0.007*"棒球隊" + 0.007*"鼓岩國小" + 0.007*"棒球" + 0.007*"運動" + 0.007*"日照中心" + 0.005*"鼓山區" + 0.005*"教育局范局長巽綠" + 0.005*"空間" + 0.005*"城市" + 0.005*"家長" + 0.005*"安全" + 0.004*"鐵路局" + 0.004*"營養午餐"

strTM =  20.003*"治安" + 0.003*"南汕巷" + 0.002*"警察機關" + 0.002*"呼叫" + 0.001*"無線電" + 0.001*"先到" + 0.001*"塞住" + 0.001*"兩輛車" + 0.001*"估價" + 0.001*"緊縮" + 0.001*"跟公" + 0.001*"住屋" + 0.001*"消防安全" + 0.001*"消防通道" + 0.001*"這片" + 0.001*"養工" + 0.001*"死角" + 0.000*"警友會" + 0.000*"戰力" + 0.000*"為民服務"

strTM =  30.021*"民政局" + 0.020*"臨水宮" + 0.018*"墳墓" + 0.016*"國產署" + 0.015*"民政局張局長乃千" + 0.009*"審計處" + 0.009*"宗教" + 0.009*"禮俗" + 0.007*"事情" + 0.007

In [17]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

pyLDAvis.enable_notebook
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict, doc_topic_dist=None)
"""
参数说明：
prepare(topic_model, corpus, dictionary, doc_topic_dist=None)
topic_model：训练得到的 gensim LdaModel 对象，不支持其他 gensim 模型类型
corpus：以词袋形式表示的语料（用于训练主题模型的语料）
dictionary：用于构建词袋模型的字典，即 gensim Dictionary 对象
doc_topic_dist：可选参数，用于传入 LDA 模型的文档主题分布，默认取值为 None，当需要多次调用 prepare 函数时，可以传入该参数
"""
pyLDAvis.display(vis)